In [1]:
from typing import TYPE_CHECKING


if TYPE_CHECKING:
    from math_rag.application.containers import ApplicationContainer
    from math_rag.infrastructure.containers import InfrastructureContainer

    application_container: ApplicationContainer
    infrastructure_container: InfrastructureContainer

In [2]:
RESET = False
%load_ext hooks.notebook_hook

2025-06-26 20:25:24,449 - INFO - datasets - config.py:54 - PyTorch version 2.6.0 available.


In [3]:
hpc_client = infrastructure_container.hpc_client()
pbs_pro_client = infrastructure_container.pbs_pro_client()

In [4]:
gpu_stats = await hpc_client.gpu_statistics()

2025-06-26 20:25:29,823 - INFO - math_rag.infrastructure.clients.ssh_client - ssh_client.py:53 - Command `gpustat | awk 'NR>=3 {print $1"_"$2"_"$3"_"$4"_"$5"_"$6"_"$7}'` in `run` returned stdout: 561561_x8000c2s4b0n1_GPU-0_0_%_5_MiB
x8000c2s4b0n1_GPU-1_0_%_5_MiB_


In [5]:
for entry in gpu_stats.entries:
    print(f'job_id={entry.job_id}')

    for sub_entry in entry.sub_entries:
        print(f'    {sub_entry}')

job_id=561561
    node='x8000c2s4b0n1' gpu='GPU-0' used_percent=0 mem_used=5242880
    node='x8000c2s4b0n1' gpu='GPU-1' used_percent=0 mem_used=5242880


In [ ]:
pushgateway_client = infrastructure_container.pushgateway_client()
prometheus_hpc_admin_client = infrastructure_container.prometheus_hpc_admin_client()

pushgateway_base_url = infrastructure_container.config.pushgateway.base_url.provided()

pbs_pro_resources_used_pusher = infrastructure_container.pbs_pro_resources_used_pusher()
gpu_stats_pusher = infrastructure_container.gpu_stats_pusher()

In [7]:
PUSHGATEWAY_JOB = 'pbs_monitor'
job_id = '561561'

In [ ]:
import asyncio


async def push():
    while True:
        # TODO use pbs_pro_resources_used_pusher and gpu_stats_pusher

        await asyncio.sleep(10)

In [ ]:
await push()

In [8]:
await pushgateway_client.delete_job(PUSHGATEWAY_JOB)

matchers = [f'pbs_job_cpu_percent{{job_id="{job_id}"}}', f'pbs_job_mem_bytes{{job_id="{job_id}"}}']
await prometheus_hpc_admin_client.delete_series(matchers)